# Reading and Writing Files with Apache Spark (PySpark)

In this notebook we will go through Read and Write various kinds of files such as Comma Separated, Tab Separated, JSON, Custom Key Value Format, Parquet.

`
@author: Anindya Saha  
@email: mail.anindya@gmail.com
`

## Table of contents

A brief synopsis of what each UDF use case is and what functionality does it touch on.

| Section                                                                             |        Demonstrates |
|:------------------------------------------------------------------------------------|:--------------------|
|[1. Creating the Spark Session](#1.-Creating-the-Spark-Session)||
|[2. Read Key Value format CSV Files](#2.-Read-Key-Value-format-CSV-Files)|Read Custom Key=Value OR Key:Value formatted files|
|[3. Read CSV Files](#3.-Read-CSV-Files)|Read Comma Separated|
|[4. Read TSV Files](#4.-Read-TSV-Files)|Read Tab Separated|
|[5. Read JSON Files](#5.-Read-JSON-Files)|Read JSON Files|
|[6. Read Parquet Files](#6.-Read-Parquet-Files)|Read Parquet Files|
|[7. Write Files](#7.-Write-Files)||

In [1]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 20)

In [3]:
# setting random seed for notebook reproducability
rnd_seed=42
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

## 1. Creating the Spark Session

In [4]:
# The following must be set in your .bashrc file
#SPARK_HOME="/home/ubuntu/spark-2.4.0-bin-hadoop2.7"
#ANACONDA_HOME="/home/ubuntu/anaconda3/envs/pyspark"
#PYSPARK_PYTHON="$ANACONDA_HOME/bin/python"
#PYSPARK_DRIVER_PYTHON="$ANACONDA_HOME/bin/python"
#PYTHONPATH="$ANACONDA_HOME/bin/python"
#export PATH="$ANACONDA_HOME/bin:$SPARK_HOME/bin:$PATH"

In [5]:
spark = (SparkSession
         .builder
         .master('local[*]')
         .appName('working-with-files')
         .getOrCreate())

In [6]:
spark

In [7]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=working-with-files>

In [8]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

## 2. Read Key Value format CSV Files

### 2.1. CSV with Key=Value Pairs

Reading a CSV file into PySpark that contains the Key=Value pairing, such that Key becomes the column and value is the data of it:

**Sample the file to understand the structure:**

In [10]:
spark.sparkContext.textFile('data/census.csv').sample(False, 0.5, rnd_seed).take(2)

['age=Middle-aged,sex=Male,education=Bachelors,native-country=United-States,race=White,marital-status=Never-married,workclass=State-gov,occupation=Adm-clerical,hours-per-week=Full-time,income=Small,capital-gain=Low,capital-loss=None',
 'age=Senior,sex=Male,education=Bachelors,native-country=United-States,race=White,marital-status=Married-civ-spouse,workclass=Self-emp-not-inc,occupation=Exec-managerial,hours-per-week=Part-time,income=Small,capital-gain=None,capital-loss=None']

**Develop the parsing logic:**

Take a sampl row and develop the tokenization strategy.

In [12]:
line='age=Middle-aged,sex=Male,education=Bachelors,native-country=United-States,race=White,marital-status=Never-married,workclass=State-gov,occupation=Adm-clerical,hours-per-week=Full-time,income=Small,capital-gain=Low,capital-loss=None'

In [13]:
[value for token in line.split(',') for key, value in [token.split('=')]]

['Middle-aged',
 'Male',
 'Bachelors',
 'United-States',
 'White',
 'Never-married',
 'State-gov',
 'Adm-clerical',
 'Full-time',
 'Small',
 'Low',
 'None']

**Create a Parse Line Function. Both these functions are equivalent:**

In [14]:
def parse_census(line):
    try:
        return [token.split('=')[1] for token in line.split(',')]
    except:
        return [] # handling malformed records

In [15]:
def parse_census(line):
    try:
        return [value for token in line.split(',') for key, value in [token.split('=')]]
    except:
        return [] # handling malformed records

In [17]:
census_rdd = spark.sparkContext.textFile('data/census.csv').map(parse_census).filter(lambda record: len(record) > 0)

In [18]:
census_rdd.take(2)

[['Middle-aged',
  'Male',
  'Bachelors',
  'United-States',
  'White',
  'Never-married',
  'State-gov',
  'Adm-clerical',
  'Full-time',
  'Small',
  'Low',
  'None'],
 ['Senior',
  'Male',
  'Bachelors',
  'United-States',
  'White',
  'Married-civ-spouse',
  'Self-emp-not-inc',
  'Exec-managerial',
  'Part-time',
  'Small',
  'None',
  'None']]

In [19]:
census_df = (spark.createDataFrame(census_rdd)
             .toDF('age', 'sex', 'education', 'native-country', 'race', 'marital-status', 
                   'workclass', 'occupation', 'hours-per-week', 'income', 'capital-gain', 'capital-loss'))
census_df.cache()

DataFrame[age: string, sex: string, education: string, native-country: string, race: string, marital-status: string, workclass: string, occupation: string, hours-per-week: string, income: string, capital-gain: string, capital-loss: string]

In [20]:
census_df.printSchema()

root
 |-- age: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- education: string (nullable = true)
 |-- native-country: string (nullable = true)
 |-- race: string (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- hours-per-week: string (nullable = true)
 |-- income: string (nullable = true)
 |-- capital-gain: string (nullable = true)
 |-- capital-loss: string (nullable = true)



In [21]:
census_df.limit(5).toPandas()

,age,sex,education,native-country,race,marital-status,workclass,occupation,hours-per-week,income,capital-gain,capital-loss
0,Middle-aged,Male,Bachelors,United-States,White,Never-married,State-gov,Adm-clerical,Full-time,Small,Low,None
1,Senior,Male,Bachelors,United-States,White,Married-civ-spouse,Self-emp-not-inc,Exec-managerial,Part-time,Small,None,None
2,Middle-aged,Male,HS-grad,United-States,White,Divorced,Private,Handlers-cleaners,Full-time,Small,None,None
3,Senior,Male,11th,United-States,Black,Married-civ-spouse,Private,Handlers-cleaners,Full-time,Small,None,None
4,Middle-aged,Female,Bachelors,Cuba,Black,Married-civ-spouse,Private,Prof-specialty,Full-time,Small,None,None


### 2.2. CSV with Key:Value Pairs With Uneven Column Colunt

Reading a CSV file into PySpark that contains the Key:Value pairing, such that Key becomes the column and value is the data of it; And not all Rows contain all the Keys:

In [22]:
spark.sparkContext.textFile('data/students.csv').sample(False, 0.5, rnd_seed).take(2)

['name:Pradnya,IP:100.0.0.4,college:SDM,year:2018',
 'name:Ram,IP:100.10.10.5,college:BVB,semester:IV,year:2018']

**Create a Parse Line Function:**

If we know all field names and keys/values do not contain embedded delimiters. then we can probably convert the key/value lines into Row object through RDD's map function.

https://stackoverflow.com/questions/50579452/reading-a-csv-file-into-pyspark-that-contains-the-keyvalue-pairing-such-that-k

In [23]:
from pyspark.sql import Row

# define a list of all field names
columns = ['name', 'IP', 'college', 'semester', 'year']

# set Row object
def parse_row(line):
    # convert line into key/value tuples. strip spaces and lowercase the `k`
    z = dict((key.strip().lower(), value.strip().lower()) for token in line.split(',') for key, value in [ token.split(':') ])
    # make sure all columns shown in the Row object
    return Row(**dict((col.lower(), z[col.lower()] if col.lower() in z else None) for col in columns))

In [24]:
students_rdd = spark.sparkContext.textFile('data/students.csv').map(parse_row).filter(lambda record: len(record) > 0)

In [25]:
students_rdd.take(2)

[Row(college='sdm', ip='100.0.0.4', name='pradnya', semester=None, year='2018'),
 Row(college='bvb', ip='100.10.10.5', name='ram', semester='iv', year='2018')]

In [26]:
students_df = spark.createDataFrame(students_rdd).toDF('name', 'ip', 'college', 'semester', 'year')
students_df.cache()

DataFrame[name: string, ip: string, college: string, semester: string, year: string]

In [27]:
students_df.limit(5).toPandas()

,name,ip,college,semester,year
0,sdm,100.0.0.4,pradnya,None,2018
1,bvb,100.10.10.5,ram,iv,2018


## 3. Read CSV Files

**Sample the file to understand the structure:**

In [28]:
spark.sparkContext.textFile('data/appl_stock.csv').sample(False, 0.5, rnd_seed).take(5)

['Date,Open,High,Low,Close,Volume,Adj Close',
 '2010-01-04,213.429998,214.499996,212.38000099999996,214.009998,123432400,27.727039',
 '2010-01-06,214.379993,215.23,210.750004,210.969995,138040000,27.333178000000004',
 '2010-01-07,211.75,212.000006,209.050005,210.58,119282800,27.28265',
 '2010-01-12,209.18999499999998,209.76999500000002,206.419998,207.720001,148614900,26.91211']

**Infer the Schema:**

In [29]:
# Let Spark know about the header and infer the Schema types!
appl_stock = spark.read.csv('data/appl_stock.csv', inferSchema=True, header=True)

In [30]:
appl_stock.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [31]:
appl_stock.show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
o

**Provide the Schema:**

We see that the `Date` column does not have any time component so we can use `DateType()` instead of `TimestampType()`. Also the `Open`, `High`, `Low`, `Close` can be of `FloatType()` and `DoubleType()` is not needed.

In [32]:
appl_stock_schema = StructType(
                        [StructField('Date', DateType(), False),
                         StructField('Open', FloatType(), False),
                         StructField('High', FloatType(), False),
                         StructField('Low', FloatType(), False),
                         StructField('Close', FloatType(), False),
                         StructField('Volume', IntegerType(), False),
                         StructField('AdjClose', FloatType(), False)]
                    )

In [33]:
# We provide the Schema to Spark
appl_stock = spark.read.csv('data/appl_stock.csv', schema=appl_stock_schema, header=True, dateFormat='yyyy-MM-dd')

In [34]:
appl_stock.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- AdjClose: float (nullable = true)



In [35]:
appl_stock.show(5)

+----------+---------+------+------+---------+---------+---------+
|      Date|     Open|  High|   Low|    Close|   Volume| AdjClose|
+----------+---------+------+------+---------+---------+---------+
|2010-01-04|   213.43| 214.5|212.38|   214.01|123432400| 27.72704|
|2010-01-05|214.59999|215.59|213.25|214.37999|150476200|27.774977|
|2010-01-06|214.37999|215.23|210.75|   210.97|138040000|27.333178|
|2010-01-07|   211.75| 212.0|209.05|   210.58|119282800| 27.28265|
|2010-01-08|210.29999| 212.0|209.06|211.98001|111902700|27.464033|
+----------+---------+------+------+---------+---------+---------+
only showing top 5 rows



## 4. Read TSV Files

**Sample the file to understand the structure:**

In [36]:
spark.sparkContext.textFile('data/sales_info.tsv').sample(False, 0.5, rnd_seed).take(5)

['Company\tPerson\tSales',
 'GOOG\tSam\t200.0',
 'GOOG\tFrank\t340.0',
 'MSFT\tTina\t600.0',
 'APPL\tChris\t350.0']

**Infer the Schema:**

In [37]:
# Let Spark know about the header and infer the Schema types!
sales_info = spark.read.csv('data/sales_info.tsv', sep='\t', inferSchema=True, header=True)

In [38]:
sales_info.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [39]:
sales_info.show(5)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
+-------+-------+-----+
only showing top 5 rows



The infered schema looks good. We do not need to provide any schema.

## 5. Read JSON Files

**Sample the file to understand the structure:**

In [40]:
spark.sparkContext.textFile('data/people.json').sample(False, 0.5, rnd_seed).take(5)

['{"name":"Michael"}', '{"name":"Andy", "age":30}']

**Infer the Schema:**

In [41]:
# Let Spark infer the Schema
people = spark.read.json('data/people.json')

In [42]:
people.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [43]:
people.show(5)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



**Provide the Schema:**

We see that the `Age` column does not need to be long so we can use `IntegerType()` instead of `long`.

In [44]:
people_schema = StructType(
                        [StructField('age', IntegerType(), False),
                         StructField('name', StringType(), False)]
                    )

In [45]:
# We provide the Schema to Spark
people = spark.read.json('data/people.json', schema=people_schema)

In [46]:
people.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [47]:
people.show(5)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## 6. Read Parquet Files

`Apache Parquet` is a columnar storage format and hence cannot be read as textfile. It will throw gibberish character out if we attempt ot do that as shown below. Also with parquet format with do not need to tell spark to inferSchema because the schema already gets written to the parquet file along with the data.

In [48]:
# parquet file cannot be read as testfile
spark.sparkContext.textFile('data/appl_stock.parquet').sample(False, 0.5, rnd_seed).take(2)

['PAR1\x15\x00\x15�n\x15�n,\x15�\x1b\x15\x00\x15\x06\x15\x08\x1c\x18\x04',
 "C\x00\x00\x18\x04\x159\x00\x00\x16\x00\x00\x00\x00�7�\x1b\x03\x00\x00\x00�\x1b\x01\x159\x00\x00\x169\x00\x00\x179\x00\x00\x189\x00\x00\x199\x00\x00\x1c9\x00\x00\x1d9\x00\x00\x1e9\x00\x00\x1f9\x00\x00 9\x00\x00$9\x00\x00%9\x00\x00&9\x00\x00'9\x00\x00*9\x00\x00+9\x00\x00,9\x00\x00-9\x00\x00.9\x00\x0019\x00\x0029\x00\x0039\x00\x0049\x00\x0059\x00\x0089\x00\x0099\x00\x00:9\x00\x00;9\x00\x00<9\x00\x00@9\x00\x00A9\x00\x00B9\x00\x00C9\x00\x00F9\x00\x00G9\x00\x00H9\x00\x00I9\x00\x00J9\x00\x00M9\x00\x00N9\x00\x00O9\x00\x00P9\x00\x00Q9\x00\x00T9\x00\x00U9\x00\x00V9\x00\x00W9\x00\x00X9\x00\x00[9\x00\x00\\9\x00\x00]9\x00\x00^9\x00\x00_9\x00\x00b9\x00\x00c9\x00\x00d9\x00\x00e9\x00\x00f9\x00\x00i9\x00\x00j9\x00\x00k9\x00\x00l9\x00\x00p9\x00\x00q9\x00\x00r9\x00\x00s9\x00\x00t9\x00\x00w9\x00\x00x9\x00\x00y9\x00\x00z9\x00\x00{9\x00\x00~9\x00\x00\x7f9\x00\x00�9\x00\x00�9\x00\x00�9\x00\x00�9\x00\x00�9\x00\x00�9\x00\x00�9\x00\x00

**Read the Parquet File:**

In [49]:
apple_stock_parquet = spark.read.parquet('data/appl_stock.parquet')

In [50]:
apple_stock_parquet.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- AdjClose: float (nullable = true)



**Note:** Observe we provided the custom schema for apple stocks to be of type Date instead of Timestamp and that is preserved.

In [51]:
apple_stock_parquet.show(5)

+----------+---------+------+------+---------+---------+---------+
|      Date|     Open|  High|   Low|    Close|   Volume| AdjClose|
+----------+---------+------+------+---------+---------+---------+
|2010-01-04|   213.43| 214.5|212.38|   214.01|123432400| 27.72704|
|2010-01-05|214.59999|215.59|213.25|214.37999|150476200|27.774977|
|2010-01-06|214.37999|215.23|210.75|   210.97|138040000|27.333178|
|2010-01-07|   211.75| 212.0|209.05|   210.58|119282800| 27.28265|
|2010-01-08|210.29999| 212.0|209.06|211.98001|111902700|27.464033|
+----------+---------+------+------+---------+---------+---------+
only showing top 5 rows



## 7. Write Files

In [52]:
# Write Tab Separated File with Header
appl_stock.write.csv('data/appl_stock_csv', sep='\t', header=True, mode='overwrite')
#appl_stock.write.csv('data/appl_stock_csv', sep='\t', header=False, mode='overwrite')

In [53]:
# Write Comma Separated File With Header
appl_stock.write.csv('data/appl_stock_tsv', header=True, mode='overwrite')
#appl_stock.write.csv('data/appl_stock_tsv', header=False, mode='overwrite')

In [54]:
# Write JSON File
appl_stock.write.csv('data/appl_stock_json', mode='overwrite')

In [55]:
# Write Parquet File
appl_stock.write.parquet('data/appl_stock_parquet', mode='overwrite')

In [56]:
spark.stop()